In [32]:
import numpy as np
import pandas as pd

df_cmp = pd.read_csv('../outcsv/4x4-1-8x8-1.cpp/csv/intra/BasketballDrive.csv')
df_ref = pd.read_csv('../out_preciso/Preciso/BasketballDrive/intra/PrecisointraBasketballDrive.csv')


np.asarray(df_cmp.iloc[:,:5])

bdb = bdbr(np.asarray(df_ref.iloc[:,:5]), np.asarray(df_cmp.iloc[:,:5]))
bdb


0.15416041134723724

In [29]:
from scipy import integrate, interpolate
import math

def bdbr(HEVC,VVC,  yuv_sel = 1):
	HEVC = np.asarray(HEVC)
	VVC = np.asarray(VVC)
	
	HEVC = HEVC[HEVC[:,0].argsort()]
	VVC = VVC[VVC[:,0].argsort()]

	xa, ya = np.log10(HEVC[:,0]), HEVC[:,yuv_sel]
	xb, yb = np.log10(VVC[:,0]), VVC[:,yuv_sel]
	
	max_i = len(ya)
	i = 1
	while(i < max_i):
		if ya[i] < ya[i-1] or yb[i] <  yb[i-1]:
			ya = np.delete( ya,i)
			yb = np.delete( yb,i)
			xa = np.delete( xa,i)
			xb = np.delete( xb,i)
			max_i = len(ya)
		else:
			i += 1

	x_interp = [max(min(xa), min(xb)), min(max(xa),max(xb))]
	y_interp = [max(min(ya), min(yb)), min(max(ya),max(yb))]

	interp_br_a = interpolate.PchipInterpolator(ya,xa)
	interp_br_b = interpolate.PchipInterpolator(yb,xb)

	bdbr_a = integrate.quad(interp_br_a, y_interp[0], y_interp[1])[0]
	bdbr_b = integrate.quad(interp_br_b, y_interp[0], y_interp[1])[0]

	bdbr = (bdbr_b - bdbr_a) / (y_interp[1] - y_interp[0])
	bdbr = (math.pow(10., bdbr)-1)*100

	return bdbr